In [37]:
import json
import sys
from tqdm import tqdm
sys.path.append('..')
from grammar.eval.match import SemanticsMatch


def judge_retrieval_result(result):
    result['true_document_ids'] = set([int(r) for r in result['true_document_ids']])
    result['retrieved_document_ids'] = set([int(r) for r in result['retrieved_document_ids']])
    result['retrieval_judgement'] = 1 if result['true_document_ids'] == result['retrieved_document_ids'] else 0
    return result

def judge_rag_result(result, semnatics_match):
    if 'judgement' in result:
        print("Judgement already exists")
        return result
    # if result['retrieval_judgement'] == 0:
    #     result["judgement"] = "Incorrect"
    # else:
    result["judgement"] = semnatics_match.generate((result["query"], result["answer"], result["gpt_response"][0]), verbose=True)[0]
    return result


linguistic_attr = 'short'
root_dir = 'spider'
semnatics_match = SemanticsMatch.from_file(root_dir=root_dir, verbalize_attrs=linguistic_attr)
file_path = f'{root_dir}/eval_results/results_{linguistic_attr}.json'
with open(file_path) as f:
    results = json.load(f)
num_retrieval_failure = sum([result['retrieval_judgement']==0 for result in results])
print(f"Retrieval failed in {num_retrieval_failure} out of {len(results)} examples")
import time
for result in tqdm(results):
    # sleep for 20 seconds after 9 examples
    if results.index(result) % 9 == 0 and results.index(result) != 0:
        print("Sleeping for 20 seconds")
        time.sleep(20)
        print("Waking up")
    result = judge_rag_result(result, semnatics_match)
num_rag_failure = sum([result['judgement']=="Incorrect" for result in results])
print(f"RAG failed in {num_rag_failure} out of {len(results)} examples")



Retrieval failed in 251 out of 397 examples


  0%|          | 0/397 [00:00<?, ?it/s]

Judgement already exists
Judgement already exists
Judgement already exists
Judgement already exists
Judgement already exists
Judgement already exists
Judgement already exists
Judgement already exists
Judgement already exists
Sleeping for 20 seconds


  2%|▏         | 9/397 [00:04<02:55,  2.21it/s]


KeyboardInterrupt: 

In [35]:
semnatics_match.save(root_dir=f'{root_dir}', override=True)
semnatics_match.llm.gpt_usage_record.write_usage(model_name='chatgpt' )

In [36]:
# ensure json serializable
for result in results:
    result['true_document_ids'] = list(result['true_document_ids'])
    result['retrieved_document_ids'] = list(result['retrieved_document_ids'])

# save results
with open(f'{root_dir}/eval_results/results_{linguistic_attr}.json', 'w') as f:
    json.dump(results, f, indent=4)